In [ ]:
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from random import random
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix as CM
from sklearn.ensemble import AdaBoostClassifier
reader = csv.reader(open('/content/drive/MyDrive/Data/planning.csv', 'r'), delimiter=",")
reader2 = csv.reader(open('/content/drive/MyDrive/Data/abalone.csv', 'r'), delimiter=",")

def sample(M,p):
  samples=np.zeros(M)
  c=np.zeros(len(p))
  sum=0
  for i in range(len(p)):#This loop finds the cdf of the distribution
    sum+=p[i]
    c[i]=sum #cdf is the cummulation of the probabilities so we add the probabilities together 
  for i in range(M):
    x=np.random.uniform(0,1)
    for j in range(len(c)):
      if x < c[j]: # determines which range the sample belongs in
        samples[i]=j
        break
  return samples

class adaboost:
    def __init__(self, numClassifiers):
      self.numClassifiers = numClassifiers
      self.alphas = [None]*numClassifiers
      self.h = [None]*numClassifiers
    def fit(self, X, y):
      #number samples and number of features
      N, F= X.shape
      weights= np.array([1/N for i in range(N)])
      
      for m in range(self.numClassifiers):
        idx = sample(len(weights), weights)   
        idx=idx.astype(int)
        h = DecisionTreeClassifier(max_depth=2)\
                        .fit(X[idx],y[idx],sample_weight=weights).predict
        
        sum=0
        for i in range(len(weights)):  
          if (y[i] != h(X[i].reshape(1,-1))):
            sum=sum+weights[i]
              
        error = sum
              
        alpha = 0.5 *np.log((1-error)/error)
        #increase weight if we got the point wrong
        for i in range(len(weights)):
          if (y[i] != h(X[i].reshape(1,-1))):
            weights[i]=weights[i]*np.exp(-1*alpha*1)
          else:
             weights[i]=weights[i]*np.exp(-1*alpha*0)
    
              
        self.h[m] = h
        self.alphas[m]=alpha

    def predict(self,X):
        y = 0
        for m in range(self.numClassifiers):
            h=self.h[m]
            y += self.alphas[m]*h(X)
        A = np.zeros(len(y))
        for i in range(len(y)):
          if y[i]!=0:
            A[i]= abs(y[i])/y[i]
          else:
            A[i]=1
        y = np.where(A==-1,-1,1)
        return y
       
def compare(x,y):
  clf = adaboost(numClassifiers=5)
  clf.fit(x,y)
  y_pred = clf.predict(x)
  print("Accuracy of my AdaBoost: %0.2f %%" % (100*sum(y_pred==y)/len(y)))
  clf2 = AdaBoostClassifier(n_estimators=5)
  clf2.fit(x,y)
  y_pred2 = clf2.predict(x)
  print("Accuracy of Sklearn: %0.2f %%" % (100*sum(y_pred2==y)/len(y)))

  
a = list(reader)
data = np.array(a).astype(float)
x=data[:, 0:12] 
y=data[:,12].astype(int)
y = np.where(y==0,-1,1)

print("Planning Dataset:")
compare(x,y)

a = list(reader2)
data = np.array(a).astype(float)
x=data[:, 0:8] 
y=data[:,8].astype(int)
y = np.where(y==0,-1,1)

print("Abalone Dataset:")
compare(x,y)

Planning Dataset:
Accuracy of my AdaBoost: 71.43 %
Accuracy of Sklearn: 73.63 %
Abalone Dataset:
Accuracy of my AdaBoost: 82.38 %
Accuracy of Sklearn: 82.93 %
